In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import pandas as pd
from tensorflow.keras.layers import LeakyReLU
import plotly.graph_objs as go

gdp=pd.read_excel("/content/data2.xls")
unemp=pd.read_excel("/content/data.xlsx")
unemp

In [ ]:
# Extracting the GDP values
gdp_values = gdp['GDP change'].values.reshape(-1, 1)
# Normalize GDP data
scaler = MinMaxScaler(feature_range=(-5,3))  # Allow for negative values
gdp_scaled = scaler.fit_transform(gdp_values)

# Define the number of previous years to consider for prediction
window_size = 10

# Create sequences and labels
sequences = []
labels = []
for i in range(len(gdp_scaled) - window_size):
    sequences.append(gdp_scaled[i:i + window_size])
    labels.append(gdp_scaled[i + window_size])

sequences = np.array(sequences)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Reshape sequences for CNN input (samples, window_size, 1)
X_train = X_train.reshape(X_train.shape[0], window_size, 1)
X_test = X_test.reshape(X_test.shape[0], window_size, 1)

# Define the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, input_shape=(window_size, 1)),
    LeakyReLU(alpha=0.1),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3),
    LeakyReLU(alpha=0.1),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(units=128),
    LeakyReLU(alpha=0.1),
    Dense(units=64),
    LeakyReLU(alpha=0.1),
    Dense(units=1, activation='linear')  # Use linear activation for flexibility
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Predict GDP for the next 10 years
last_sequence = sequences[-1].reshape(1, window_size, 1)
future_gdp_scaled = []

no_of_years_to_predict=30
# Use the model to predict GDP
for i in range(no_of_years_to_predict):
    prediction = model.predict(last_sequence)
    future_gdp_scaled.append(prediction)
    last_sequence = np.concatenate([last_sequence[:, 1:, :], prediction.reshape(1, 1, 1)], axis=1)

future_gdp_scaled = np.array(future_gdp_scaled).reshape(-1)
predicted_gdp = scaler.inverse_transform(future_gdp_scaled.reshape(-1, 1))


# Plotting the results using Plotly
years = gdp['Year']
actual_trace = go.Scatter(x=years, y=gdp_values.flatten(), mode='lines', name='Actual GDP')
predicted_trace = go.Scatter(x=list(range(max(years), max(years) + no_of_years_to_predict)), y=predicted_gdp.flatten(), mode='lines', name='Predicted GDP')

layout = go.Layout(title=f'Predicted GDP of a Country for the Next {no_of_years_to_predict}  Years',
                   xaxis=dict(title='Year'),
                   yaxis=dict(title='GDP'))

fig = go.Figure(data=[actual_trace, predicted_trace], layout=layout)
fig.show()
predicted_trace

NameError: name 'gdp' is not defined

In [ ]:

# Extracting the GDP values
gdp_values = gdp['GDP change'].values.reshape(-1, 1)
# Normalize GDP data
scaler = MinMaxScaler(feature_range=(-5, 3))  # Allow for negative values
gdp_scaled = scaler.fit_transform(gdp_values)

# Define the number of previous years to consider for prediction
window_size = 30

# Create sequences and labels
sequences = []
labels = []
for i in range(len(gdp_scaled) - window_size):
    sequences.append(gdp_scaled[i:i + window_size])
    labels.append(gdp_scaled[i + window_size])

sequences = np.array(sequences)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.3, random_state=72)

# Reshape sequences for CNN input (samples, window_size, 1)
X_train = X_train.reshape(X_train.shape[0], window_size, 1)
X_test = X_test.reshape(X_test.shape[0], window_size, 1)

# Define the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, input_shape=(window_size, 1)),
    LeakyReLU(alpha=0.5),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3),
    LeakyReLU(alpha=0.8),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(units=128),
    LeakyReLU(alpha=0.5),
    Dense(units=64),
    LeakyReLU(alpha=0.8),
    Dense(units=1, activation='linear')  # Use linear activation for flexibility
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Predict GDP for the next 10 years
last_sequence = sequences[-1].reshape(1, window_size, 1)
future_gdp_scaled = []

# Use the model to predict GDP
for i in range(50):
    prediction = model.predict(last_sequence)
    future_gdp_scaled.append(prediction)
    last_sequence = np.concatenate([last_sequence[:, 1:, :], prediction.reshape(1, 1, 1)], axis=1)

future_gdp_scaled = np.array(future_gdp_scaled).reshape(-1)
predicted_gdp = scaler.inverse_transform(future_gdp_scaled.reshape(-1, 1))

# Plotting the results using Plotly
years = gdp['Year']
actual_trace = go.Scatter(x=years, y=gdp_values.flatten(), mode='lines', name='Actual GDP')
predicted_trace = go.Scatter(x=list(range(max(years), max(years) + 50)), y=predicted_gdp.flatten(), mode='lines', name='Predicted GDP')

layout = go.Layout(title='Predicted GDP of a Country for the Next 10 Years',
                   xaxis=dict(title='Year'),
                   yaxis=dict(title='GDP'))

fig = go.Figure(data=[actual_trace, predicted_trace], layout=layout)
predicted_trace
fig.show()

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 1.5914 - val_loss: 4.0590
Epoch 2/100
1/1 [==============================] - 0s 41ms/step - loss: 1.1823 - val_loss: 4.4156
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 1.1492 - val_loss: 3.9662
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 0.8969 - val_loss: 3.4555
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 0.7118 - val_loss: 3.1383
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 0.6420 - val_loss: 3.0065
Epoch 7/100
1/1 [==============================] - 0s 40ms/step - loss: 0.5919 - val_loss: 3.0102
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 0.4954 - val_loss: 3.1068
Epoch 9/100
1/1 [==============================] - 0s 42ms/step - loss: 0.3768 - val_loss: 3.2677
Epoch 10/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2924 - val_loss: 3.4166
Epoch 11/100
1/1 [===

In [ ]:

# Extracting the unemp values
unemp_values = unemp['Unemp'].values.reshape(-1, 1)
# Normalize unemp data
scaler = MinMaxScaler(feature_range=(-1,1))  # Allow for negative values
unemp_scaled = scaler.fit_transform(unemp_values)

# Define the number of previous years to consider for prediction
window_size = 15

# Create sequences and labels
sequences = []
labels = []
for i in range(len(unemp_scaled) - window_size):
    sequences.append(unemp_scaled[i:i + window_size])
    labels.append(unemp_scaled[i + window_size])

sequences = np.array(sequences)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Reshape sequences for CNN input (samples, window_size, 1)
X_train = X_train.reshape(X_train.shape[0], window_size, 1)
X_test = X_test.reshape(X_test.shape[0], window_size, 1)

# Define the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, input_shape=(window_size, 1)),
    LeakyReLU(alpha=0.1),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3),
    LeakyReLU(alpha=0.1),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(units=128),
    LeakyReLU(alpha=0.1),
    Dense(units=64),
    LeakyReLU(alpha=0.1),
    Dense(units=1, activation='linear')  # Use linear activation for flexibility
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Predict unemp for the next 10 years
last_sequence = sequences[-1].reshape(1, window_size, 1)
future_unemp_scaled = []

# Use the model to predict unemp
for i in range(30):
    prediction = model.predict(last_sequence)
    future_unemp_scaled.append(prediction)
    last_sequence = np.concatenate([last_sequence[:, 1:, :], prediction.reshape(1, 1, 1)], axis=1)

future_unemp_scaled = np.array(future_unemp_scaled).reshape(-1)
predicted_unemp = scaler.inverse_transform(future_unemp_scaled.reshape(-1, 1))


# Plotting the results using Plotly
years = unemp['Year']
actual_trace = go.Scatter(x=years, y=unemp_values.flatten(), mode='lines', name='Actual GDP')
predicted_trace = go.Scatter(x=list(range(max(years), max(years) + 30)), y=predicted_unemp.flatten(), mode='lines', name='Predicted unemp')

layout = go.Layout(title='Predicted unemployment of a Country for the Next 30 Years',
                   xaxis=dict(title='Year'),
                   yaxis=dict(title='unemp'))

fig = go.Figure(data=[actual_trace, predicted_trace], layout=layout)
fig.show()
predicted_trace

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.2710 - val_loss: 0.2203
Epoch 2/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2117 - val_loss: 0.1759
Epoch 3/100
1/1 [==============================] - 0s 42ms/step - loss: 0.1662 - val_loss: 0.1350
Epoch 4/100
1/1 [==============================] - 0s 44ms/step - loss: 0.1259 - val_loss: 0.0989
Epoch 5/100
1/1 [==============================] - 0s 43ms/step - loss: 0.0918 - val_loss: 0.0656
Epoch 6/100
1/1 [==============================] - 0s 37ms/step - loss: 0.0632 - val_loss: 0.0388
Epoch 7/100
1/1 [==============================] - 0s 37ms/step - loss: 0.0437 - val_loss: 0.0225
Epoch 8/100
1/1 [==============================] - 0s 42ms/step - loss: 0.0371 - val_loss: 0.0189
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 0.0441 - val_loss: 0.0233
Epoch 10/100
1/1 [==============================] - 0s 40ms/step - loss: 0.0568 - val_loss: 0.0267
Epoch 11/100
1/1 [===

Scatter({
    'mode': 'lines',
    'name': 'Predicted unemp',
    'x': [2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035,
          2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047,
          2048, 2049, 2050, 2051, 2052, 2053],
    'y': array([-0.1852291 ,  0.05515546, -0.08832169,  0.04220419,  0.1886032 ,
                 0.10342755,  0.16843566,  0.04595038,  0.03568675,  0.04832347,
                 0.02550934, -0.03377867,  0.00619138,  0.00304281,  0.01562286,
                 0.00589702,  0.01720154,  0.04572767,  0.02770742,  0.02194771,
                 0.0144229 , -0.00846554, -0.002557  , -0.02957028, -0.03417602,
                -0.03952044, -0.04374765, -0.03798385, -0.03959228, -0.03861592],
               dtype=float32)
})